In [5]:
import numpy as np
import math
import time
import random

# Veri dosyasını oku
def read_tsp_data(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
    
    n = int(lines[0].strip())
    points = []
    for i in range(1, n + 1):
        x, y = map(float, lines[i].strip().split())
        points.append((x, y))
    
    return n, points

# İki nokta arasındaki mesafe
def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Toplam yol uzunluğunu hesapla
def calculate_total_distance(route, points):
    total = 0
    for i in range(len(route) - 1):
        total += distance(points[route[i]], points[route[i + 1]])
    return total

# Rastgele bir başlangıç rotası oluştur
def create_initial_solution(n):
    solution = list(range(n))
    random.shuffle(solution)
    solution.append(solution[0])  # Turun kapanması için başlangıca dön
    return solution

# Benzetimli Tavlama algoritması
def simulated_annealing(points, initial_temp=1000, cooling_rate=0.995, stopping_temp=1e-8, stopping_iter=100000):
    n = len(points)
    current_solution = create_initial_solution(n)
    best_solution = current_solution.copy()
    current_distance = calculate_total_distance(current_solution, points)
    best_distance = current_distance
    
    temperature = initial_temp
    iteration = 1
    
    while temperature > stopping_temp and iteration < stopping_iter:
        # Komşu çözüm üret (2-opt swap)
        new_solution = current_solution[:-1].copy()
        i, j = sorted(random.sample(range(n), 2))
        new_solution[i:j + 1] = reversed(new_solution[i:j + 1])
        new_solution.append(new_solution[0])  # tekrar başa dön
        
        new_distance = calculate_total_distance(new_solution, points)
        
        if new_distance < current_distance:
            current_solution = new_solution
            current_distance = new_distance
            if new_distance < best_distance:
                best_solution = new_solution.copy()
                best_distance = new_distance
        else:
            delta = new_distance - current_distance
            acceptance_probability = math.exp(-delta / temperature)
            if random.random() < acceptance_probability:
                current_solution = new_solution
                current_distance = new_distance
        
        temperature *= cooling_rate
        iteration += 1

        if iteration % 10000 == 0:
            print(f"İterasyon {iteration}: Sıcaklık = {temperature:.4f}, Anlık En İyi Mesafe = {best_distance:.2f}")
    
    return best_solution, best_distance

# En yakın komşu algoritması
def nearest_neighbor(points):
    n = len(points)
    visited = [False] * n
    route = [0]
    visited[0] = True

    for _ in range(n - 1):
        last = route[-1]
        next_point = -1
        min_dist = float('inf')
        for i in range(n):
            if not visited[i]:
                dist = distance(points[last], points[i])
                if dist < min_dist:
                    min_dist = dist
                    next_point = i
        route.append(next_point)
        visited[next_point] = True
    
    route.append(route[0])  # başladığı düğüme dön
    return route

# Ana program
def main():
    filename = "tsp_150.txt"
    
    print("TSP verisi okunuyor...")
    n, points = read_tsp_data(filename)
    print(f"Toplam {n} nokta okundu.")
    
    print("En Yakın Komşu algoritması ile başlangıç çözümü oluşturuluyor...")
    start_time = time.time()
    nn_route = nearest_neighbor(points)
    nn_distance = calculate_total_distance(nn_route, points)
    nn_time = time.time() - start_time
    print(f"En Yakın Komşu çözümü: Mesafe = {nn_distance:.2f}, Süre = {nn_time:.2f} saniye")
    
    print("Benzetimli Tavlama algoritması başlatılıyor...")
    start_time = time.time()
    best_route, best_distance = simulated_annealing(points, initial_temp=1000, cooling_rate=0.995)
    sa_time = time.time() - start_time
    print(f"Benzetimli Tavlama çözümü: Mesafe = {best_distance:.2f}, Süre = {sa_time:.2f} saniye")
    
    improvement = ((nn_distance - best_distance) / nn_distance) * 100
    print(f"İyileştirme: %{improvement:.2f}")
    
    print("En iyi rota:")
    print(best_route)

if __name__ == "__main__":
    main()


TSP verisi okunuyor...
Toplam 150 nokta okundu.
En Yakın Komşu algoritması ile başlangıç çözümü oluşturuluyor...
En Yakın Komşu çözümü: Mesafe = 32084.98, Süre = 0.00 saniye
Benzetimli Tavlama algoritması başlatılıyor...
Benzetimli Tavlama çözümü: Mesafe = 52041.52, Süre = 0.32 saniye
İyileştirme: %-62.20
En iyi rota:
[111, 44, 140, 137, 27, 122, 126, 85, 74, 8, 20, 16, 93, 29, 23, 17, 86, 114, 127, 61, 98, 106, 41, 56, 100, 135, 103, 3, 87, 34, 94, 129, 62, 7, 136, 19, 57, 79, 130, 43, 117, 112, 108, 37, 32, 33, 132, 12, 72, 22, 146, 50, 102, 25, 48, 10, 18, 47, 35, 0, 53, 145, 113, 124, 15, 45, 144, 55, 46, 95, 63, 26, 97, 76, 109, 104, 142, 28, 39, 134, 133, 147, 77, 143, 131, 84, 88, 123, 78, 40, 83, 6, 2, 81, 120, 49, 66, 92, 101, 139, 80, 58, 105, 128, 116, 149, 30, 69, 14, 99, 38, 91, 82, 36, 96, 73, 121, 141, 54, 9, 75, 31, 71, 89, 138, 4, 1, 64, 13, 42, 115, 60, 21, 5, 110, 67, 65, 119, 148, 24, 107, 68, 11, 52, 118, 51, 125, 90, 70, 59, 111]
